# Makeover Monday | 2026 W6 Global Big Max Index
The data for this weeks Makeover Monday cam from the [Economist's Interactive Big Max Index article](https://www.economist.com/interactive/big-mac-index) with data compiled by the [Makeover Monday](www.makeovermonday.co.uk) team using resources from [the Economic Big Mac Data GitHub repository](https://github.com/TheEconomist/big-mac-data?tab=readme-ov-file). This data sources **Big Mac** prices from MacDonals from around the world, and uses various normalization and missing data techniques to calculate an adjusted GDP index.

 * **Article Linke** : https://www.economist.com/interactive/big-mac-index
 * **Data Source Link**: https://github.com/TheEconomist/big-mac-data?tab=readme-ov-file
 * **Compiled Data** (*published by Makeover Monday*): https://data.world/makeovermonday/2026w6-global-big-mac-index

In [ ]:
import pandas as pd
import pandas_gbq
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates # Required for date formatting
import seaborn as sns
from IPython.display import HTML, display

In [ ]:
df = pd.read_csv('https://query.data.world/s/gmcntqa3nxudymry7ktnoqnsqf5pdv?dws=00000')

## Exploratory Data Analysis

In [ ]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
date,1894,42,1/1/2022,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iso_a3,1894,56,ARG,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
currency_code,1894,57,ARS,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
name,1894,57,Argentina,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
local_price,1894.0,NaN,NaN,NaN,17699.407027,399889.928299,1.05,8.0,28.5,139.75,16020000.0
dollar_ex,1894.0,NaN,NaN,NaN,5188.829613,102072.316578,0.3008,3.0743,7.80045,52.3199,3613989.071
dollar_price,1894.0,NaN,NaN,NaN,3.467605,1.318122,0.639966,2.469535,3.299551,4.29,9.081066
USD_raw,1894.0,NaN,NaN,NaN,-0.160225,0.311021,-0.84515,-0.385728,-0.21202,0.0,1.53472
EUR_raw,1894.0,NaN,NaN,NaN,-0.237531,0.263413,-0.83471,-0.432227,-0.273355,-0.082318,0.87063
GBP_raw,1894.0,NaN,NaN,NaN,-0.185552,0.288548,-0.85151,-0.39448,-0.228775,-0.01142,1.13764


In [ ]:
df.dtypes

,0
date,object
iso_a3,object
currency_code,object
name,object
local_price,float64
dollar_ex,float64
dollar_price,float64
USD_raw,float64
EUR_raw,float64
GBP_raw,float64


According to the **Codebook** section of the Big Mac Data [README](https://github.com/TheEconomist/big-mac-data?tab=readme-ov-file) file, the files that we have are:

<font size="4" face="Arial">
  <table>
      <tr>
          <td align="left"><b>variable<b></td>
          <td align="left"><b>definition<b></td>
          <td align="left"><b>source<b></td>
      </tr>
      <tr>
          <td align="left">date</td>
          <td align="left">Date of observation</td>
          <td align="left" class="empty-cell"></td>
      </tr>
      <tr>
          <td align="left">iso_a3</td>
          <td align="left">Three-character ISO 3166-1 country code</td>
          <td align="left" class="empty-cell"></td>
      </tr>
      <tr>
          <td align="left">currency_code</td>
          <td align="left">Three-character ISO 4217 currency code</td>
          <td align="left" class="empty-cell"></td>
      </tr>
      <tr>
          <td align="left">name</td>
          <td align="left">Country name</td>
          <td align="left" class="empty-cell"></td>
      </tr>
      <tr>
          <td align="left">local_price</td>
          <td align="left">Price of a Big Mac in the local currency</td>
          <td align="left">McDonalds; The Economist</td>
      </tr>
      <tr>
          <td align="left">dollar_ex</td>
          <td align="left">Local currency units per dollar</td>
          <td align="left">Reuters</td>
      </tr>
      <tr>
          <td align="left">dollar_price</td>
          <td align="left">Price of a Big Mac in dollars	</td>
          <td align="left" class="empty-cell"></td>
      </tr>
      <tr>
          <td align="left">CURR*_raw</td>
          <td align="left">Raw index, relative to the Currency Code*</td>
          <td align="left">Multiple Fields </td>
      </tr>
      <tr>
          <td align="left">GDP_bigmac</td>
          <td align="left">GDP per person, in dollars</td>
          <td align="left">IMF? potential mapped to GDP_dollar</td>
      </tr>
      <tr>
          <td align="left">adj_price</td>
          <td align="left">	GDP-adjusted price of a Big Mac, in dollars</td>
        <td align="left" class="empty-cell"></td>
      </tr>
      <tr>
          <td align="left">CURR*_adjusted</td>
          <td align="left">Multiple Fields Adjusted index, relative to the Currency Code*</td>
          <td align="left">Multiple Fields</td>
      </tr>
  </table>
</font>


**Note on ISO vs Currency Code/Name**
For some reason we have 57 unique countries/currency_codes and yet only 56 unique ISO currency codes. For this reason, I will ignore iso_a3 and work with currency_code and name.

In [ ]:
df = df.drop(columns=['iso_a3'])

## Clean Step: Fixing the date field

In [ ]:
df['date'].min()

Timestamp('2000-04-01 00:00:00')

In [ ]:
df['date'].max()

Timestamp('2025-01-01 00:00:00')

The max/top date is 7/9/2019 and the "earliest"/min date appears to be 2006 - but the Date field is an object - so we should convert that to a date or date time field to really ensure we can do any time series.

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
date_range = df['date'].max() - df['date'].min()
print(date_range)

9041 days 00:00:00


## Clean Step: Handling Missing Data

In [ ]:
df.isnull().sum()

,0
date,0
currency_code,0
name,0
local_price,0
dollar_ex,0
dollar_price,0
USD_raw,0
EUR_raw,0
GBP_raw,0
JPY_raw,0


We have 249 missing fields for all of the adjusted values - and when looking at the missing data we can see that its for approximately 21 countries, most likely due to missing IMF GDP data. I will not drop these columns for now.

In [ ]:
missingdata_df = df[df['GDP_bigmac'].isnull()]
missingdata_df.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
date,249,NaN,NaN,NaN,2015-06-12 21:58:33.253011968,2000-04-01 00:00:00,2012-01-01 00:00:00,2017-01-01 00:00:00,2019-07-09 00:00:00,2025-01-01 00:00:00,NaN
currency_code,249,21,UAH,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
name,249,21,Venezuela,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
local_price,249.0,NaN,NaN,NaN,77183.750141,1.05,13.0,87.0,1130.0,16020000.0,1016145.24503
dollar_ex,249.0,NaN,NaN,NaN,17785.54563,0.3025,4.29465,24.8,500.83,3613989.071,229158.045978
dollar_price,249.0,NaN,NaN,NaN,3.238922,0.664323,2.328323,3.244006,3.935147,9.081066,1.228206
USD_raw,249.0,NaN,NaN,NaN,-0.226423,-0.84515,-0.40595,-0.24781,-0.10086,1.25337,0.27911
EUR_raw,249.0,NaN,NaN,NaN,-0.285799,-0.83471,-0.47711,-0.30873,-0.14634,0.86163,0.25233
GBP_raw,249.0,NaN,NaN,NaN,-0.223981,-0.85151,-0.42166,-0.22549,-0.07996,1.13764,0.279647
JPY_raw,249.0,NaN,NaN,NaN,-0.018943,-0.78698,-0.22448,-0.03807,0.13405,1.58427,0.347048


In [ ]:
missingdata_df['name'].unique()

array(['Euro area', 'Venezuela', 'Costa Rica', 'Sri Lanka', 'Ukraine',
       'Uruguay', 'UAE', 'Vietnam', 'United Arab Emirates', 'Azerbaijan',
       'Bahrain', 'Guatemala', 'Honduras', 'Jordan', 'Kuwait', 'Lebanon',
       'Moldova', 'Nicaragua', 'Oman', 'Qatar', 'Romania'], dtype=object)

# Save to BigQuery Table
Save the df to a dedicated table in my BigQuery warehouse. I will then use SQL to get the final perfect data set before before pushing it to Google Sheets that I will use to connect to Tableau Public.

In [ ]:

#convert the df to target table in bigquery dataet
UPLOAD_TO_BQ = False # set True when you actually want to write tables

if UPLOAD_TO_BQ:
  project_id = 'your_project_id'
  destination_table = 'your_dataset.another_new_table'

  pandas_gbq.to_gbq(
      dataframe=df,
      destination_table=destination_table,
      project_id=project_id,
      if_exists='replace' ## 'if_exists' options: 'fail', 'replace', 'append'
  )

100%|██████████| 1/1 [00:00<00:00, 4236.67it/s]


# SQL Query to Finalize the Dataset for Tableau
*note: this will not run in any python/notebook

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
WITH bigmaxindex AS (
  SELECT
    date,
    EXTRACT(YEAR FROM date) AS year,
    EXTRACT(MONTH FROM date) AS month,
    currency_code,
    (CASE
      WHEN name = "UAE" THEN "United Arab Emirates" -- found that duplicate name mismatch is iso_a3
      ELSE name
    END) AS name,
    local_price,
    dollar_ex,
    dollar_price
  FROM `data-projects-478723.makeover_monday.bigmacindex`
),

country_region AS (
  -- https://corporate.mcdonalds.com/content/dam/sites/corp/nfl/pdf/20231202_Investor%20Update%20Fact%20Sheet.pdf
  SELECT * FROM UNNEST([

    -- North America
      STRUCT("United States" AS name, "North America"  AS region_group),
      STRUCT("Canada", "North America"),

    -- Western Europe
      STRUCT("Britain", "Western Europe" ),
      STRUCT("Switzerland", "Western Europe"),
      STRUCT("Norway", "Western Europe"),
      STRUCT("Sweden", "Western Europe"),
      STRUCT("Denmark", "Western Europe"),
      STRUCT("Euro area", "Western Europe"),

    -- Eastern Europe
      STRUCT("Czech Republic", "Eastern Europe"),
      STRUCT("Poland", "Eastern Europe"),
      STRUCT("Hungary", "Eastern Europe"),
      STRUCT("Romania", "Eastern Europe"),
      STRUCT("Moldova", "Eastern Europe"),
      STRUCT("Ukraine", "Eastern Europe"),
      STRUCT("Russia", "Eastern Europe"),
      STRUCT("Turkey", "Eastern Europe"),

    -- Asia
      STRUCT("Japan", "Asia"),
      STRUCT("South Korea","Asia"),
      STRUCT("Singapore", "Asia"),
      STRUCT("Taiwan", "Asia"),
      STRUCT("Hong Kong", "Asia"),

      STRUCT("China", "Asia"),
      STRUCT("India", "Asia"),
      STRUCT("Indonesia", "Asia"),
      STRUCT("Malaysia", "Asia"),
      STRUCT("Philippines", "Asia"),
      STRUCT("Thailand", "Asia"),
      STRUCT("Vietnam", "Asia"),
      STRUCT("Pakistan", "Asia"),
      STRUCT("Sri Lanka", "Asia"),

      STRUCT("Australia", "Asia"),
      STRUCT("New Zealand", "Asia"),


    -- Middle East and Africa
      STRUCT("United Arab Emirates","Middle East and Africa"),
      STRUCT("Kuwait", "Middle East and Africa"),
      STRUCT("Qatar", "Middle East and Africa"),
      STRUCT("Oman", "Middle East and Africa"),
      STRUCT("Saudi Arabia", "Middle East and Africa"),
      STRUCT("Bahrain", "Middle East and Africa"),
      STRUCT("Lebanon", "Middle East and Africa"),
      STRUCT("Jordan", "Middle East and Africa"),
      STRUCT("Israel", "Middle East and Africa"),
      STRUCT("Egypt", "Middle East and Africa"),
      STRUCT("South Africa", "Middle East and Africa"),
      STRUCT("Azerbaijan", "Middle East and Africa"),

    -- Latin America
      STRUCT("Argentina", "Latin America"),
      STRUCT("Brazil", "Latin America"),
      STRUCT("Chile", "Latin America"),
      STRUCT("Colombia", "Latin America"),
      STRUCT("Mexico", "Latin America"),
      STRUCT("Peru", "Latin America"),
      STRUCT("Uruguay", "Latin America"),
      STRUCT("Venezuela", "Latin America"),
      STRUCT("Costa Rica", "Latin America"),
      STRUCT("Guatemala", "Latin America"),
      STRUCT("Honduras", "Latin America"),
      STRUCT("Nicaragua", "Latin America")

  ])
),

usbmi_monthly AS (
  SELECT
    year,
    month,
    currency_code,
    name,
    AVG(dollar_price) AS average_monthly_dollar_price
  FROM bigmaxindex
  WHERE name = "United States"
  GROUP BY year, month, currency_code, name
)

SELECT
  bmi.date,
  bmi.year,
  bmi.month,
  bmi.currency_code,
  bmi.name,
  COALESCE(cr.region_group, "Unmapped") AS region_group,
  bmi.local_price,
  bmi.dollar_ex,
  bmi.dollar_price, -- this is what we will compare against,
  us_bmi.average_monthly_dollar_price,
  SAFE_DIVIDE(bmi.dollar_price,us_bmi.average_monthly_dollar_price) AS relative_price, -- closer to 1 is the same as the US
  AVG(bmi.dollar_price) OVER (
    PARTITION BY
    bmi.year, bmi.month, cr.region_group
  ) AS region_avg_dollar_price,
  SAFE_DIVIDE(
    bmi.dollar_price,
    AVG(bmi.dollar_price) OVER (
      PARTITION BY bmi.year, bmi.month, cr.region_group
    )
  ) AS relative_price_region

FROM bigmaxindex bmi
  LEFT JOIN usbmi_monthly us_bmi
  ON bmi.year = us_bmi.year
  AND bmi.month = us_bmi.month

  LEFT JOIN country_region cr
    ON bmi.name = cr.name

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

# Tableau Dashboard
The final dashboard has been published to Tableau, exploring the [Global Big Mac Prices Relative to the United States and Region](https://public.tableau.com/app/profile/allison.jones/viz/MoM2026_w6BigMacIndex/BigMacIndexAccrosstheWorld).

This was a labor of love because I wanted to be able to see each region separately and give a different insight for each Region:

![Western Europe](images/mom_2026_w6/tableau_dashboard_mom_2026_w4_western_europe.png)

![Eastern Europe](images/mom_2026_w6/tableau_dashboard_mom_2026_w4_eastern_europe.png)